In [1]:
import os 
import glob
import time
import warnings
import numpy as np
from joblib import Parallel, delayed

from datasets import *
from benchmarks import *

In [2]:
reps = 10
num_cores = 10 # number of cores for parallel computing
nterms_max = 20 # maximum number of ridge terms
slfn_nterms_list = [10,30,50,100,200]
aim_nterms_list = [3,5,7,10,20]

knot_num = 10
datanum = int(1e4)

reg_lambda = [0.1, 0.5, 0.9]
reg_gamma = "GCV"
knot_dist = 'quantile'

d_seq = [20]

In [3]:
data_set_list = ['Case1','Case2',
                 'Case3','Case4',
                 'Case5','Case6']
data_loader_list = [data_generator1,data_generator2,
                    data_generator3,data_generator4,
                    data_generator5,data_generator6]

In [4]:
results_train_latex_list = []
results_test_latex_list = []
results_tune_time_latex_list = [] # time including tuning cost
results_optparam_time_latex_list = [] # time for the optimal set of hyper-parameters
results_ortho_latex_list = []
results_nterms_latex_list = []

for d in d_seq:

    all_results_tune_time = []
    all_results_optparam_time = []
    all_results_ortho = []
    all_results_train = []
    all_results_test = []
    all_results_nterms = []
    
    for idx, data_loader in enumerate(data_loader_list[:1]): # only use the first data generator for illustration

        vb = data_set_list[idx]
        _, _, train_x, test_x, train_y, test_y, task_type, meta_info, get_metric = data_loader(datanum=datanum,d=d,
                                                                                               rand_seed=0)
        print('---------------------- Dimension:',d,'DataSet:',vb,'----------------------')
        
        folder = "./results/DiffDim/" + str(d) + '/' + vb + "/"
        
        if not os.path.exists(folder):
            os.makedirs(folder)

        start = time.time()
        if not os.path.exists(folder + 'seqstein_nonortho_stat.npy'):
            stat = Parallel(n_jobs=num_cores)(delayed(batch_parallel)("SeqStein", data_loader, 
                                                                                       random_state=random_state, 
                                                                                       data_type='simulation',
                                                                                       d=d,datanum=datanum,
                                                                                       knot_num=knot_num,
                                                                                       reg_lambda = reg_lambda,
                                                                                       ortho_enhance=[False],
                                                                                       nterms=nterms_max) 
                                                               for task_id, random_state in enumerate(range(reps)))
            seqstein_nonortho_stat = pd.concat(stat).values
            np.save(folder + 'seqstein_nonortho_stat.npy', seqstein_nonortho_stat)
        print("SeqStein NonOrtho Finished!", "Time Cost ", np.round(time.time() - start, 2), " Seconds!")
        
        if not os.path.exists(folder + 'seqstein_ortho_stat.npy'):
            stat = Parallel(n_jobs=num_cores)(delayed(batch_parallel)("SeqStein", data_loader, 
                                                                                       random_state=random_state, 
                                                                                       data_type='simulation',
                                                                                       d=d,datanum=datanum,
                                                                                       knot_num=knot_num,
                                                                                       reg_lambda = reg_lambda,
                                                                                       ortho_enhance=[True],
                                                                                       nterms=nterms_max) 
                                                               for task_id, random_state in enumerate(range(reps)))
            seqstein_ortho_stat = pd.concat(stat).values
            np.save(folder + 'seqstein_ortho_stat.npy', seqstein_ortho_stat)
        print("SeqStein Ortho Finished!", "Time Cost ", np.round(time.time() - start, 2), " Seconds!")
        
        if not os.path.exists(folder + 'aimlow_stat.npy'):
            stat = Parallel(n_jobs=num_cores)(delayed(batch_parallel)("AIM", data_loader, 
                                                                      d=d,datanum=datanum,
                                                                              knot_dist=knot_dist,
                                                                              data_type='simulation',
                                                                              random_state=random_state,
                                                                              knot_num=knot_num, optlevel='low',
                                                                              nterms=aim_nterms_list) 
                                             for task_id, random_state in enumerate(range(reps)))
            aimlow_stat = pd.concat(stat).values
            np.save(folder + 'aimlow_stat.npy', aimlow_stat)
        print("AIMLow Finished!", "Time Cost ", np.round(time.time() - start, 2), " Seconds!")

        if not os.path.exists(folder + 'aimhigh_stat.npy'):
            stat = Parallel(n_jobs=num_cores)(delayed(batch_parallel)("AIM", data_loader, 
                                                                              knot_dist=knot_dist,
                                                                              d=d,datanum=datanum,
                                                                              data_type='simulation',
                                                                              random_state=random_state,
                                                                              knot_num=knot_num, optlevel='high',
                                                                              nterms=aim_nterms_list) 
                                             for task_id, random_state in enumerate(range(reps)))
            aimhigh_stat = pd.concat(stat).values
            np.save(folder + 'aimhigh_stat.npy', aimhigh_stat)
        print("AIMHigh Finished!", "Time Cost ", np.round(time.time() - start, 2), " Seconds!")

        if not os.path.exists(folder + 'slfn_stat.npy'):
            stat = Parallel(n_jobs=num_cores)(delayed(batch_parallel)("SLFN", data_loader,
                                                                      data_type='simulation',
                                                                      d=d,datanum=datanum,
                                                                       random_state=random_state, 
                                                                       nterms=slfn_nterms_list) 
                                         for task_id, random_state in enumerate(range(reps)))
            slfn_stat = pd.concat(stat).values
            np.save(folder + 'slfn_stat.npy', slfn_stat)
        print("SLFN Finished!", "Time Cost ", np.round(time.time() - start, 2), " Seconds!")
        
        if not os.path.exists(folder + 'exnn_stat.npy'):
            try:
                stat = Parallel(n_jobs=num_cores)(delayed(batch_parallel)("ExNN", data_loader,
                                                                          data_type='simulation',
                                                                          d=d,datanum=datanum,
                                                                           random_state=random_state, 
                                                                           nterms=nterms_max) 
                                             for task_id, random_state in enumerate(range(reps)))
            except:
                stat = Parallel(n_jobs=3)(delayed(batch_parallel)("ExNN", data_loader,
                                                                          data_type='simulation',
                                                                          d=d,datanum=datanum,
                                                                           random_state=random_state, 
                                                                           nterms=nterms_max) 
                                             for task_id, random_state in enumerate(range(reps)))
            exnn_stat = pd.concat(stat).values
            np.save(folder + 'exnn_stat.npy', exnn_stat)
        print("ExNN Finished!", "Time Cost ", np.round(time.time() - start, 2), " Seconds!")
        
        if not os.path.exists(folder + 'mlp_stat.npy'):
            stat = Parallel(n_jobs=num_cores)(delayed(batch_parallel)("MLP", data_loader,
                                                                      data_type='simulation',
                                                                      d=d,datanum=datanum,
                                                                       random_state=random_state) 
                                         for task_id, random_state in enumerate(range(reps)))
            mlp_stat = pd.concat(stat).values
            np.save(folder + 'mlp_stat.npy', mlp_stat)
        print("MLP Finished!", "Time Cost ", np.round(time.time() - start, 2), " Seconds!")
        
        
        seqstein_nonortho_stat = np.load(folder + 'seqstein_nonortho_stat.npy')
        seqstein_ortho_stat = np.load(folder + 'seqstein_ortho_stat.npy')
        aimlow_stat = np.load(folder + 'aimlow_stat.npy')
        aimhigh_stat = np.load(folder + 'aimhigh_stat.npy')
        slfn_stat = np.load(folder + 'slfn_stat.npy')
        mlp_stat = np.load(folder + 'mlp_stat.npy')
        exnn_stat = np.load(folder + 'exnn_stat.npy') 
        print('Load the Savings.')
        
        with warnings.catch_warnings():
            print('Generate the statistics.')
            warnings.simplefilter("ignore", category=RuntimeWarning)
            stat = pd.DataFrame({"SeqStein-NonOrtho-Mean":np.nanmean(np.vstack(seqstein_nonortho_stat), 0),
                                 "SeqStein-Ortho-Mean":np.nanmean(np.vstack(seqstein_ortho_stat), 0),
                                 "AIMLow-Mean":np.nanmean(np.vstack(aimlow_stat), 0),
                                 "AIMHigh-Mean":np.nanmean(np.vstack(aimhigh_stat), 0),
                                 "SLFN-Mean":np.nanmean(np.vstack(slfn_stat), 0),
                                 "ExNN-Mean":np.nanmean(np.vstack(exnn_stat), 0),
                                 "MLP-Mean":np.nanmean(np.vstack(mlp_stat), 0),
                                 "SeqStein-NonOrtho-Std":np.nanstd(np.vstack(seqstein_nonortho_stat), 0),
                                 "SeqStein-Ortho-Std":np.nanstd(np.vstack(seqstein_ortho_stat), 0),
                                 "AIMLow-Std":np.nanstd(np.vstack(aimlow_stat), 0),
                                 "AIMHigh-Std":np.nanstd(np.vstack(aimhigh_stat), 0),
                                 "SLFN-Std":np.nanstd(np.vstack(slfn_stat), 0),
                                 "ExNN-Std":np.nanstd(np.vstack(exnn_stat), 0),
                                 "MLP-Std":np.nanstd(np.vstack(mlp_stat), 0)},
                                 index=["train_metric", "validation_metric", "test_metric", 
                                        'alltune_time_cost','optparam_time_cost','ortho_measure', 
                                       'nterms']).T
    
        results_train = stat[["train_metric"]].T
        results_train["Dataset"] = vb
        results_train["#Samples"] = train_x.shape[0] + test_x.shape[0]
        results_train["#Features"] = train_x.shape[1]
        results_train["Task"] = 'Regression'
        results_train = results_train.set_index("Dataset")

        results_test = stat[["test_metric"]].T
        results_test["Dataset"] = vb
        results_test["#Samples"] = train_x.shape[0] + test_x.shape[0]
        results_test["#Features"] = train_x.shape[1]
        results_test["Task"] = 'Regression'
        results_test = results_test.set_index("Dataset")

        results_tune_time = stat[["alltune_time_cost"]].T
        results_tune_time["Dataset"] = vb
        results_tune_time["#Samples"] = train_x.shape[0] + test_x.shape[0]
        results_tune_time["#Features"] = train_x.shape[1]
        results_tune_time["Task"] = 'Time'
        results_tune_time = results_tune_time.set_index("Dataset")   
        
        results_optparam_time = stat[["optparam_time_cost"]].T
        results_optparam_time["Dataset"] = vb
        results_optparam_time["#Samples"] = train_x.shape[0] + test_x.shape[0]
        results_optparam_time["#Features"] = train_x.shape[1]
        results_optparam_time["Task"] = 'Time'
        results_optparam_time = results_optparam_time.set_index("Dataset")   

        results_ortho = stat[["ortho_measure"]].T
        results_ortho["Dataset"] = vb
        results_ortho["#Samples"] = train_x.shape[0] + test_x.shape[0]
        results_ortho["#Features"] = train_x.shape[1]
        results_ortho["Task"] = 'Ortho'
        results_ortho = results_ortho.set_index("Dataset")  

        results_nterms = stat[['nterms']].T
        results_nterms["Dataset"] = vb
        results_nterms["#Samples"] = train_x.shape[0] + test_x.shape[0]
        results_nterms["#Features"] = train_x.shape[1]
        results_nterms["Task"] = 'Nterms'
        results_nterms = results_nterms.set_index("Dataset")  
        
        all_results_tune_time.append(results_tune_time)
        all_results_optparam_time.append(results_optparam_time)
        all_results_train.append(results_train)
        all_results_test.append(results_test)
        all_results_ortho.append(results_ortho)
        all_results_nterms.append(results_nterms)
        print('Generate and append the results for data sets.')
        
    results_train_latex = gen_reg_latex_results(pd.concat(all_results_train), 3)  
    results_test_latex = gen_reg_latex_results(pd.concat(all_results_test), 3)
    results_tune_time_latex = gen_reg_latex_results(pd.concat(all_results_tune_time), 3)
    results_optparam_time_latex = gen_reg_latex_results(pd.concat(all_results_optparam_time), 3)
    results_ortho_latex = gen_reg_latex_results(pd.concat(all_results_ortho), 3)
    results_nterms_latex = gen_reg_latex_results(pd.concat(all_results_nterms), 3)

    results_train_latex_list.append(results_train_latex)
    results_test_latex_list.append(results_test_latex)
    results_tune_time_latex_list.append(results_tune_time_latex)
    results_optparam_time_latex_list.append(results_optparam_time_latex)
    results_ortho_latex_list.append(results_ortho_latex)
    results_nterms_latex_list.append(results_nterms_latex)
    print('Generate and append the results for dimension.')

---------------------- Dimension: 20 DataSet: Case1 ----------------------
SeqStein NonOrtho Finished! Time Cost  43.43  Seconds!
SeqStein Ortho Finished! Time Cost  62.59  Seconds!
AIMLow Finished! Time Cost  103.78  Seconds!
AIMHigh Finished! Time Cost  811.89  Seconds!
SLFN Finished! Time Cost  1966.85  Seconds!
ExNN Finished! Time Cost  5328.02  Seconds!


/home/r13user2/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


MLP Finished! Time Cost  5587.02  Seconds!
Load the Savings.
Generate the statistics.
Generate and append the results for data sets.
Generate and append the results for dimension.


In [5]:
# training error
results_train_df = pd.concat(results_train_latex_list).reset_index().set_index(['#Features','Dataset'])
results_train_df.to_csv('./results/test_train_diffd.csv')
results_train_df

,,SeqStein-NonOrtho,SeqStein-Ortho,AIMLow,AIMHigh,SLFN,MLP,ExNN,#Samples
#Features,Dataset,,,,,,,,
20,Case1,1.075$\pm$0.012,1.111$\pm$0.017,1.248$\pm$0.178,$\mathbf{0.994}$$\pm$0.008,1.021$\pm$0.012,1.001$\pm$0.016,1.003$\pm$0.007,10000


In [6]:
# test error
results_test_df = pd.concat(results_test_latex_list).reset_index().set_index(['#Features','Dataset'])
results_test_df.to_csv('./results/test_test_diffd.csv')
results_test_df

,,SeqStein-NonOrtho,SeqStein-Ortho,AIMLow,AIMHigh,SLFN,MLP,ExNN,#Samples
#Features,Dataset,,,,,,,,
20,Case1,1.081$\pm$0.018,1.113$\pm$0.031,1.271$\pm$0.191,1.014$\pm$0.016,1.048$\pm$0.025,1.058$\pm$0.017,$\mathbf{1.005}$$\pm$0.016,10000


In [7]:
# orthogonal measure
results_ortho_df = pd.concat(results_ortho_latex_list).reset_index().set_index(['#Features','Dataset'])
results_ortho_df.to_csv('./results/test_ortho_diffd.csv')
results_ortho_df

,,SeqStein-NonOrtho,SeqStein-Ortho,AIMLow,AIMHigh,SLFN,MLP,ExNN,#Samples
#Features,Dataset,,,,,,,,
20,Case1,0.362$\pm$0.086,0.118$\pm$0.023,0.292$\pm$0.080,0.297$\pm$0.073,0.295$\pm$0.049,nan$\pm$nan,$\mathbf{0.002}$$\pm$0.001,10000


In [8]:
# time (include tuning cost)
results_tune_time_df = pd.concat(results_tune_time_latex_list).reset_index().set_index(['#Features','Dataset'])
results_tune_time_df.to_csv('./results/test_tune_time_diffd.csv')
results_tune_time_df

,,SeqStein-NonOrtho,SeqStein-Ortho,AIMLow,AIMHigh,SLFN,MLP,ExNN,#Samples
#Features,Dataset,,,,,,,,
20,Case1,19.534$\pm$6.068,$\mathbf{13.426}$$\pm$3.063,35.322$\pm$3.039,596.153$\pm$49.707,1076.541$\pm$44.402,200.640$\pm$31.722,2211.552$\pm$529.453,10000


In [9]:
# time (for optimal set of hyper-parameters)
results_optparam_time_df = pd.concat(results_optparam_time_latex_list).reset_index().set_index(['#Features','Dataset'])
results_optparam_time_df.to_csv('./results/test_optparam_time_diffd.csv')
results_optparam_time_df

,,SeqStein-NonOrtho,SeqStein-Ortho,AIMLow,AIMHigh,SLFN,MLP,ExNN,#Samples
#Features,Dataset,,,,,,,,
20,Case1,19.534$\pm$6.068,13.426$\pm$3.063,$\mathbf{10.850}$$\pm$4.315,261.933$\pm$180.767,151.385$\pm$63.244,200.640$\pm$31.722,282.718$\pm$71.286,10000


In [10]:
# number of ridge terms
results_nterms_df = pd.concat(results_nterms_latex_list).reset_index().set_index(['#Features','Dataset'])
results_nterms_df.to_csv('./results/test_nterms_diffd.csv')
results_nterms_df

,,SeqStein-NonOrtho,SeqStein-Ortho,AIMLow,AIMHigh,SLFN,MLP,ExNN,#Samples
#Features,Dataset,,,,,,,,
20,Case1,11.000$\pm$4.266,7.600$\pm$2.289,14.200$\pm$5.980,14.700$\pm$5.367,31.000$\pm$56.648,nan$\pm$nan,$\mathbf{4.800}$$\pm$1.249,10000
